## Project 2 milestone, CS-401 : Applied data analysis, group : TheAvengers

#### import useful libraries and python files

In [49]:
import pandas as pd
import numpy as np
import json
import sys
import requests
import re

from data_loader import *
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Lets first download our datasets and  extract the former countriers, the genres and the languages from each movie and create new columns feature that will be able to access more easily on the mentionned datas.

In [2]:
#loading the datasets
dataFrame_movie = load_cmu_movies()
dataFrame_character = load_cmu_characters()
#dataFrame_kaggle_movie = load_kaggle_movies()

#extracting the aforementioned features
dataFrame_movie['Country_List'] = dataFrame_movie['Countries'].apply(extract_features_names)
dataFrame_movie['Genre_List'] = dataFrame_movie['Genres'].apply(extract_features_names)
dataFrame_movie['Language_List'] = dataFrame_movie['Language'].apply(extract_features_names)

We decided to add additional datasets in order to complete our datas. However, trying to merge those datasets based on their Movies names doesn't work as there are movies that have the same Name but are in fact different. Therefore, it is complicated to differentiate movies based on their names. To overcome this issue, we identified each movie by its Freebase ID or its IMDB ID (depending on the dataset). To do so, we first need to retreive the IMDB ID (i.e. tconst column) based on the movie's Freebase ID. The code to do this can be retreive below but it has been put in comments as the file generated contains data directly retreive from the wiki-data and therefore may vary over time (if I run the below function in few days, I could have more or less rows)

In [ ]:

#df_id_translation = get_wikidata_id_translations()
#display(df_id_translation)

#save the dataframe to a JSON file
#df_id_translation.to_json('id-translation.wikidata.json', orient='records', lines=True)

Instead, we can load a file that has been already generate by the above function "get_wikidata_id_translations()" :

In [3]:
#replace 'id-translation.wikidata.json' with the path to your JSON file
file_path = 'id-translation.wikidata.json'

#load the JSON file into a DataFrame
df_id_translation = pd.read_json(file_path, orient='records', lines=True)

#display the loaded DataFrame
display(df_id_translation)

,tconst,Freebase ID
0,nm3628633,/m/0k83_1r
1,nm0012078,/m/02q173g
2,nm0394250,/m/03kfdk
3,tt0324924,/m/0ct2z8
4,nm0119354,/m/0356cy
...,...,...
990453,tt29521500,None
990454,nm2106941,None
990455,nm2186144,None
990456,nm4531538,None


Then, we can drop Nan and duplicates values and merge it with the CMU movies dataset 

In [4]:
df_id_translation = df_id_translation.dropna(subset=['Freebase ID'])
df_id_translation.drop_duplicates(subset=['Freebase ID'], keep='first', inplace=True)

dataFrame_movie = pd.merge(dataFrame_movie, df_id_translation, on='Freebase ID', how='left')

print(dataFrame_movie.shape)

(81740, 16)


In [5]:
#compute percentage of missing values for df_mvoei
percentage_missing_values_year = (dataFrame_movie['Year'].isna().sum()/len(dataFrame_movie['Year']))*100
print(f"The percentage of missing values 'Year' release date is {format(percentage_missing_values_year, '.3f')}%.")

percentage_missing_values_month = (dataFrame_movie['Month'].isna().sum()/len(dataFrame_movie['Month']))*100
print(f"The percentage of missing values 'Month' release date (and therefore 'Day') is {format(percentage_missing_values_month, '.3f')}%.")

percentage_missing_values_revenues = (dataFrame_movie['Revenue'].isna().sum()/len(dataFrame_movie['Revenue']))*100
print(f"The percentage of missing values 'Box office' is {format(percentage_missing_values_revenues, '.3f')}%.")

#percentage_missing_values_runtime = (dataFrame_movie['Runtime'].isna().sum()/len(dataFrame_movie['Runtime']))*100
#print(f"The percentage of missing values 'Runtime' is {format(percentage_missing_values_runtime, '.3f')}%.")

percentage_missing_values_tconst = (dataFrame_movie['tconst'].isna().sum()/len(dataFrame_movie['tconst']))*100
print(f"The percentage of missing values 'tconst' is {format(percentage_missing_values_tconst, '.3f')}%.")

The percentage of missing values 'Year' release date is 8.444%.
The percentage of missing values 'Month' release date (and therefore 'Day') is 51.833%.
The percentage of missing values 'Box office' is 89.724%.
The percentage of missing values 'tconst' is 9.769%.


### Load addtionnal datasets and merge what we need 

In [6]:
#load imdb dataset (from kaggle)
dataFrame_imdb_movie = load_movie_imdb_kaggle()
dataFrame_imdb_rating = load_rating_imdb_kaggle() 

In [7]:
#merge movies with rating 
df_movie_rating = pd.merge(dataFrame_imdb_movie, dataFrame_imdb_rating, on='tconst', how='inner')
df_movie_rating.head()

,tconst,titleType,Name,originalTitle,isAdult,Year,endYear,Runtime,genres,averageRating,numVotes
0,tt0000009,movie,Miss Jerry,Miss Jerry,0,1894,\N,45.0,Romance,5.3,207
1,tt0000147,movie,The Corbett-Fitzsimmons Fight,The Corbett-Fitzsimmons Fight,0,1897,\N,100.0,"Documentary,News,Sport",5.3,484
2,tt0000502,movie,Bohemios,Bohemios,0,1905,\N,100.0,\N,4.1,15
3,tt0000574,movie,The Story of the Kelly Gang,The Story of the Kelly Gang,0,1906,\N,70.0,"Action,Adventure,Biography",6.0,854
4,tt0000591,movie,The Prodigal Son,L'enfant prodigue,0,1907,\N,90.0,Drama,5.0,21


In [8]:
#merge CMU dataset with IMDB dataset
dataFrame_movie = pd.merge(dataFrame_movie, df_movie_rating[['tconst', 'averageRating', 'numVotes']], on=['tconst'], how='left')
print(dataFrame_movie.shape)

(81740, 18)


In [9]:
#load kaggle movies dataset
dataFrame_kaggle_movie = load_kaggle_movies()

In [10]:
#merging the two datasets
dataFrame_movie = dataFrame_movie.merge(dataFrame_kaggle_movie[['Name', 'tconst', 'Revenue']], on=['Name', 'tconst'], how='left', suffixes=('', '_df2'))

#use 'combine_first' to fill in the 'Revenue' values from df_kaggle_movie where they are NaN in df_movie
dataFrame_movie['Revenue'] = dataFrame_movie['Revenue'].combine_first(dataFrame_movie['Revenue_df2'])

#drop the 'Revenue_df2' column
dataFrame_movie.drop('Revenue_df2', axis=1, inplace=True)

#drop duplicates
dataFrame_movie = dataFrame_movie.drop_duplicates(subset='Wikipedia ID', keep='first')
display(dataFrame_movie)

,Wikipedia ID,Freebase ID,Name,Release date,Revenue,Run Time,Language,Countries,Genres,Year,Month,Day,Country_List,Genre_List,Language_List,tconst,averageRating,numVotes
0,3196793,/m/08yl5d,Getting Away with Murder: The JonBenét Ramsey ...,2000-02-16,NaN,95.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/02n4kr"": ""Mystery"", ""/m/03bxz7"": ""Biograp...",2000,2,16,[United States of America],"[Mystery, Biographical film, Drama, Crime Drama]",[English Language],tt0245916,NaN,NaN
1,28463795,/m/0crgdbh,Brun bitter,1988,NaN,83.0,"{""/m/05f_3"": ""Norwegian Language""}","{""/m/05b4w"": ""Norway""}","{""/m/0lsxr"": ""Crime Fiction"", ""/m/07s9rl0"": ""D...",1988,<NA>,<NA>,[Norway],"[Crime Fiction, Drama]",[Norwegian Language],tt0094806,5.6,40.0
2,9363483,/m/0285_cd,White Of The Eye,1987,NaN,110.0,"{""/m/02h40lc"": ""English Language""}","{""/m/07ssc"": ""United Kingdom""}","{""/m/01jfsb"": ""Thriller"", ""/m/0glj9q"": ""Erotic...",1987,<NA>,<NA>,[United Kingdom],"[Thriller, Erotic thriller, Psychological thri...",[English Language],tt0094320,6.1,2888.0
3,261236,/m/01mrr1,A Woman in Flames,1983,NaN,106.0,"{""/m/04306rv"": ""German Language""}","{""/m/0345h"": ""Germany""}","{""/m/07s9rl0"": ""Drama""}",1983,<NA>,<NA>,[Germany],[Drama],[German Language],tt0083949,6.0,621.0
4,13696889,/m/03cfc81,The Gangsters,1913-05-29,NaN,35.0,"{""/m/06ppq"": ""Silent film"", ""/m/02h40lc"": ""Eng...","{""/m/09c7w0"": ""United States of America""}","{""/m/02hmvc"": ""Short Film"", ""/m/06ppq"": ""Silen...",1913,5,29,[United States of America],"[Short Film, Silent film, Indie, Comedy]","[Silent film, English Language]",tt0002894,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81752,35228177,/m/0j7hxnt,Mermaids: The Body Found,2011-03-19,NaN,120.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/07s9rl0"": ""Drama""}",2011,3,19,[United States of America],[Drama],[English Language],tt1816585,NaN,NaN
81753,34980460,/m/0g4pl34,Knuckle,2011-01-21,0.0,96.0,"{""/m/02h40lc"": ""English Language""}","{""/m/03rt9"": ""Ireland"", ""/m/07ssc"": ""United Ki...","{""/m/03bxz7"": ""Biographical film"", ""/m/07s9rl0...",2011,1,21,"[Ireland, United Kingdom]","[Biographical film, Drama, Documentary]",[English Language],tt1606259,6.8,3191.0
81754,9971909,/m/02pygw1,Another Nice Mess,1972-09-22,NaN,66.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/06nbt"": ""Satire"", ""/m/01z4y"": ""Comedy""}",1972,9,22,[United States of America],"[Satire, Comedy]",[English Language],tt0362411,5.8,110.0
81755,913762,/m/03pcrp,The Super Dimension Fortress Macross II: Lover...,1992-05-21,NaN,150.0,"{""/m/03_9r"": ""Japanese Language""}","{""/m/03_3d"": ""Japan""}","{""/m/06n90"": ""Science Fiction"", ""/m/0gw5n2f"": ...",1992,5,21,[Japan],"[Science Fiction, Japanese Movies, Adventure, ...",[Japanese Language],tt0113726,NaN,NaN


In [12]:
percentage_missing_values_revenues = (dataFrame_movie['Revenue'].isna().sum()/len(dataFrame_movie['Revenue']))*100
print(f"The percentage of missing values 'Revenue' is {format(percentage_missing_values_revenues, '.3f')}%.")

percentage_missing_values_rating = (dataFrame_movie['averageRating'].isna().sum()/len(dataFrame_movie['averageRating']))*100
print(f"The percentage of missing values 'averageRating' is {format(percentage_missing_values_rating, '.3f')}%.")


The percentage of missing values 'Revenue' is 70.124%.
The percentage of missing values 'averageRating' is 31.271%.


Now, we want to regroup all actors that have played on the same movie and add this new features to our dataframe.

In [17]:
display(dataFrame_character)

,Wikipedia ID,Freebase ID,Release date,Character Name,Actor Birth,Actor Gender,Actor Height,Ethnicity,Actor Name,Actor Age At Movie Release,Freebase Actor/Character Map ID,Freebase Character ID,Freebase Actor ID
0,975900,/m/03vyhn,2001-08-24,Lieutenant Melanie Ballard,1974-08-15,F,1.780,/m/044038p,Natasha Henstridge,27.0,/m/0jys3m,/m/0bgchn4,/m/0346l4
1,975900,/m/03vyhn,2001-08-24,Desolation Williams,1969-06-15,M,1.727,/m/0x67,Ice Cube,32.0,/m/0jys3g,/m/0bgchn_,/m/01vw26l
2,975900,/m/03vyhn,2001-08-24,Sgt Jericho Butler,1967-09-12,M,1.750,NaN,Jason Statham,33.0,/m/02vchl6,/m/0bgchnq,/m/034hyc
3,975900,/m/03vyhn,2001-08-24,Bashira Kincaid,1977-09-25,F,1.650,NaN,Clea DuVall,23.0,/m/02vbb3r,/m/0bgchp9,/m/01y9xg
4,975900,/m/03vyhn,2001-08-24,Commander Helena Braddock,1949-05-26,F,1.727,/m/0x67,Pam Grier,52.0,/m/02vdcfp,/m/0bgchnd,/m/0418ft
...,...,...,...,...,...,...,...,...,...,...,...,...,...
450663,913762,/m/03pcrp,1992-05-21,Elensh,1970-05,F,NaN,NaN,Dorothy Elias-Fahn,NaN,/m/0kr406c,/m/0kr406h,/m/0b_vcv
450664,913762,/m/03pcrp,1992-05-21,Hibiki,1965-04-12,M,NaN,NaN,Jonathan Fahn,27.0,/m/0kr405_,/m/0kr4090,/m/0bx7_j
450665,28308153,/m/0cp05t9,1957,NaN,1941-11-18,M,1.730,/m/02w7gg,David Hemmings,15.0,/m/0g8ngmc,NaN,/m/022g44
450666,28308153,/m/0cp05t9,1957,NaN,NaN,NaN,NaN,NaN,Roberta Paterson,NaN,/m/0g8ngmj,NaN,/m/0g8ngmm


In [26]:
#group dataFrame_character by Freebase ID and list the characters 
dataFrame_actor = dataFrame_character.groupby('Freebase ID')['Actor Name'].apply(list).reset_index()

dataFrame_actor.columns = ['Freebase ID', 'List of actors']

display(dataFrame_actor)



,Freebase ID,List of actors
0,/m/011_mj,"[nan, Charlie Chaplin, John Rand, Steve Murphy..."
1,/m/011_p6,"[Fay Wray, Eugenie Besserer, Richard Arlen, Ge..."
2,/m/011_zy,"[Ralph Forbes, Alice Joyce, H. B. Warner, Geor..."
3,/m/011wtv,"[Tom Cruise, Max von Sydow, Steve Harris, Neal..."
4,/m/011x_4,"[Rick Ducommun, Marita Geraghty, Brian Doyle-M..."
...,...,...
64325,/m/0yyn5,"[Melanie Griffith, Harrison Ford, Sigourney We..."
64326,/m/0yyts,"[Esther Rolle, Morgan Freeman, Jessica Tandy, ..."
64327,/m/0yzbg,"[Alexandra Powers, Robin Williams, Robert Sean..."
64328,/m/0yzp8,"[Amy Madigan, Gaby Hoffmann, Ray Liotta, Timot..."


In [38]:
#that is the way to access the list of actors for a specific movie
#print(dataFrame_actor[dataFrame_actor['Freebase ID'] == '/m/011_mj']['List of actors'].iloc[0])

In [41]:
#now lets merge this new dataframe with the DataFrame_movie
dataFrame_movie = dataFrame_movie.merge(dataFrame_actor, on='Freebase ID', how='left')
display(dataFrame_movie)

,Wikipedia ID,Freebase ID,Name,Release date,Revenue,Run Time,Language,Countries,Genres,Year,Month,Day,Country_List,Genre_List,Language_List,tconst,averageRating,numVotes,List of actors
0,3196793,/m/08yl5d,Getting Away with Murder: The JonBenét Ramsey ...,2000-02-16,NaN,95.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/02n4kr"": ""Mystery"", ""/m/03bxz7"": ""Biograp...",2000,2,16,[United States of America],"[Mystery, Biographical film, Drama, Crime Drama]",[English Language],tt0245916,NaN,NaN,"[Allen Cutler, Alice Barrett, Rod Britt, Hildy..."
1,28463795,/m/0crgdbh,Brun bitter,1988,NaN,83.0,"{""/m/05f_3"": ""Norwegian Language""}","{""/m/05b4w"": ""Norway""}","{""/m/0lsxr"": ""Crime Fiction"", ""/m/07s9rl0"": ""D...",1988,<NA>,<NA>,[Norway],"[Crime Fiction, Drama]",[Norwegian Language],tt0094806,5.6,40.0,"[Frank Krog, Kristin Kajander, Vidar Sandem, A..."
2,9363483,/m/0285_cd,White Of The Eye,1987,NaN,110.0,"{""/m/02h40lc"": ""English Language""}","{""/m/07ssc"": ""United Kingdom""}","{""/m/01jfsb"": ""Thriller"", ""/m/0glj9q"": ""Erotic...",1987,<NA>,<NA>,[United Kingdom],"[Thriller, Erotic thriller, Psychological thri...",[English Language],tt0094320,6.1,2888.0,"[David Keith, Cathy Moriarty]"
3,261236,/m/01mrr1,A Woman in Flames,1983,NaN,106.0,"{""/m/04306rv"": ""German Language""}","{""/m/0345h"": ""Germany""}","{""/m/07s9rl0"": ""Drama""}",1983,<NA>,<NA>,[Germany],[Drama],[German Language],tt0083949,6.0,621.0,"[Gudrun Landgrebe, Mathieu Carrière, Hanns Zis..."
4,13696889,/m/03cfc81,The Gangsters,1913-05-29,NaN,35.0,"{""/m/06ppq"": ""Silent film"", ""/m/02h40lc"": ""Eng...","{""/m/09c7w0"": ""United States of America""}","{""/m/02hmvc"": ""Short Film"", ""/m/06ppq"": ""Silen...",1913,5,29,[United States of America],"[Short Film, Silent film, Indie, Comedy]","[Silent film, English Language]",tt0002894,NaN,NaN,[Roscoe Arbuckle]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81735,35228177,/m/0j7hxnt,Mermaids: The Body Found,2011-03-19,NaN,120.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/07s9rl0"": ""Drama""}",2011,3,19,[United States of America],[Drama],[English Language],tt1816585,NaN,NaN,NaN
81736,34980460,/m/0g4pl34,Knuckle,2011-01-21,0.0,96.0,"{""/m/02h40lc"": ""English Language""}","{""/m/03rt9"": ""Ireland"", ""/m/07ssc"": ""United Ki...","{""/m/03bxz7"": ""Biographical film"", ""/m/07s9rl0...",2011,1,21,"[Ireland, United Kingdom]","[Biographical film, Drama, Documentary]",[English Language],tt1606259,6.8,3191.0,NaN
81737,9971909,/m/02pygw1,Another Nice Mess,1972-09-22,NaN,66.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/06nbt"": ""Satire"", ""/m/01z4y"": ""Comedy""}",1972,9,22,[United States of America],"[Satire, Comedy]",[English Language],tt0362411,5.8,110.0,"[Rich Little, Herb Voland]"
81738,913762,/m/03pcrp,The Super Dimension Fortress Macross II: Lover...,1992-05-21,NaN,150.0,"{""/m/03_9r"": ""Japanese Language""}","{""/m/03_3d"": ""Japan""}","{""/m/06n90"": ""Science Fiction"", ""/m/0gw5n2f"": ...",1992,5,21,[Japan],"[Science Fiction, Japanese Movies, Adventure, ...",[Japanese Language],tt0113726,NaN,NaN,"[Isshin Chiba, Tsutomu Takayama, 笠原弘子, Yumi Tō..."


Finally, we can take a look on the percentage of movies for which the list of actors is not available :

In [51]:
#lets first make a copy of the dataset 
test = dataFrame_movie.copy()

#we define a function on the data_loader file that allows us to check if a list is empty or not
test['Non-Empty Characters'] = test['List of actors'].apply(is_nonempty_list)

#compute the percentage of movies with empty or NaN character lists
percentage_with_nonempty_characters = (test[test['Non-Empty Characters']].shape[0] / test.shape[0]) * 100


print(f"The percentage of missing values 'List of actors' is {format(100 - percentage_with_nonempty_characters, '.3f')}%.")

The percentage of missing values 'List of actors' is 22.212%.


## Let's start with the analysis

We can see that we managed to decrease the pourcentage of missing values. Now that we have now our 2 dataframes. First, We will drop the NA in the revenue. We need to have the revenues of the movie we are working with. 

In [ ]:
dataFrame_movie = dataFrame_movie.dropna(subset=['Revenue'])

First, we are counting the number of actor per film (actor = character) 

In [ ]:
df_count = dataFrame_movie.merge(dataFrame_character, on = 'Wikipedia ID', how = 'left') # POPULATE EACH MOVIE WITH ACTOR 
counter = df_count.groupby('Wikipedia ID')['Actor Name'].nunique() # We count the number of actor per movie

In [ ]:
df_count

In [ ]:
import matplotlib.pyplot as plt


plt.hist(counter, bins =max(counter))

plt.title('Frequency of the Number of actor per movie')
plt.xlabel('Number of actor per movie')
plt.ylabel('Frequency')
plt.show()


Now, the idea is to create a matrix. Each row of the matrix represent a differente movie. Every Column represents a different actor. The matrix returns 1 if the actor i plays in the movie j and 0 otherwise

In [ ]:
# Create a binary matrix where rows represent movies, columns represent actors
binary_matrix = df_count.pivot_table(index='Name', columns='Actor Name', aggfunc='size', fill_value=0)

# Reset the index to have movie name as a column
binary_matrix = binary_matrix.reset_index()

# Fill NaN values with 0 (if necessary)
binary_matrix = binary_matrix.fillna(0)

# Rename the columns with a prefix for clarity
binary_matrix.columns = ['Name'] + [ str(col) for col in binary_matrix.columns[1:]]

# Set 'movie name' as the index 
binary_matrix.set_index('Name', inplace=True)

In [ ]:
binary_matrix

We have to much actor right now, therefore we have to get rid of many of them

Using this matrix, we will remove the actors that appears in less than 35 movies 

In [ ]:
actor_sum=binary_matrix.sum()
actors_to_drop=actor_sum[actor_sum<35].index

In [ ]:
filtered_binary_matrix=binary_matrix.drop(actors_to_drop,axis=1)


In [ ]:
filtered_binary_matrix  

In [ ]:
for i in filtered_binary_matrix.columns.tolist():
    print(i)

Even with this filter, we still have a high number of actor (113). This is still a high number and if we want to test every pair of actor as a independant variable, we need (113*112)/2 beta. We still have the possibility to filter more using maybe more qualitative aspect. 

An important aspect that we have to care about is the interaction between actors. We build now the matrix of interraction between actors with more than 25 films. (Why 25? because it is better to have a bigger number of actor) 

In [ ]:
actors_to_drop=actor_sum[actor_sum<25].index

filtered_binary_matrix=binary_matrix.drop(actors_to_drop,axis=1)

filtered_binary_matrix


For each column of the filtered_binary_matrix, we multiply by every other column and them sum the new column

In [ ]:
binary_matrix_column = filtered_binary_matrix.columns


matrix = np.zeros([len(binary_matrix_column),len(binary_matrix_column)])

i = 0
j = 0
# Iteration in every row and column to calculate the interaction
for col in binary_matrix_column:
    result = []
    j = 0
    for col2 in binary_matrix_column:
        #since the column are dummies, we can just multiply
        matrix[i,j]= (filtered_binary_matrix[col]* filtered_binary_matrix[col2]).sum()
        j+= 1
    i+=1

# We put the result in a dataframe
pairing_df = pd.DataFrame(matrix, columns=binary_matrix_column, index=binary_matrix_column)

# We save the result in a csv file
pairing_df.to_csv('interaction_matrix_actors.csv', index=False) 




We have our matrix of interaction. We will sort a list of actor that interract the most with his pear. We set the diagonal of the matrix equal to 0 and count the number of interaction of each actor

In [ ]:
matrix_interaction = matrix - matrix*np.eye(len(binary_matrix_column))
pure_interaction = pd.DataFrame(matrix_interaction, columns=binary_matrix_column, index=binary_matrix_column)

pure_interaction = pure_interaction.sum(axis =1)

pure_interaction.sort_values(ascending=False)


Now we have a list of actors who collaborate most frequently with their peers.


Additional Idea : We should maybe draw a map of the connection between actor to get rid of interration between actor that have never worked together. We should maybe add more women (even if their number of film is lower). We could maybe see an intresting trend
We could also use other types of variable such as the age difference between the actors

In [ ]:
#let's try to create a graph network
import networkx as nx
import matplotlib.pyplot as plt

#create a mapping from actor names to numbers (for better visualization)
actor_to_number = {actor: num for num, actor in enumerate(pairing_df.columns)}

#create a graph
G = nx.from_pandas_adjacency(pairing_df)
G = nx.relabel_nodes(G, actor_to_number)

In [ ]:
#lets fix a treshold to remove the edges with a low interaction in order to have a better visualization
threshold = 50 #we can change this value to see the difference  
for u, v, d in list(G.edges(data=True)):
    if d['weight'] <= threshold:
        G.remove_edge(u, v)

In [ ]:
#plot the graph
import matplotlib.pyplot as plt

pos = nx.spring_layout(G, scale=2)  # positions for all nodes
#weights = [G[u][v]['weight'] for u, v in G.edges()]
edge_widths = [d['weight'] for u, v, d in G.edges(data=True)]  

#nx.draw(G, pos, edges=G.edges(), width=weights, with_labels=True, node_size=50, font_size=8)
nx.draw(G, pos, width=edge_widths, with_labels=True, node_size=70, font_size=12, alpha=0.7)

#adjust the label position
edge_labels = nx.get_edge_attributes(G, 'weight')
nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_size=5)

plt.show()